In [1]:
# Uninstall any opencv packages first (from current env)
!pip uninstall -y opencv-python opencv-contrib-python

# Then install into your user site-packages (no admin)
!pip install --user opencv-contrib-python


Found existing installation: opencv-contrib-python 4.12.0.88
Uninstalling opencv-contrib-python-4.12.0.88:
  Successfully uninstalled opencv-contrib-python-4.12.0.88


  Using cached opencv_contrib_python-4.12.0.88-cp37-abi3-win_amd64.whl (45.3 MB)


In [2]:
import cv2

In [3]:
def generate_dataset():
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    def face_cropped(img):
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)
        # scaling factor = 1.3
        # minimum neighbor = 5
         
        if faces is ():
            return None
        for (x,y,w,h) in faces:
            cropped_face = img[y:y+h,x:x+w]
        return cropped_face
     
    cap = cv2.VideoCapture(0)
    id =1
    img_id = 0
     
    while True:
        ret,frame = cap.read()
        if face_cropped(frame) is not None:
            img_id+=1
            face = cv2.resize(face_cropped(frame), (200,200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            file_name_path = "data/user."+str(id)+"."+str(img_id)+".jpg"
            cv2.imwrite(file_name_path, face)
            cv2.putText(face, str(img_id), (50,50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            
            cv2.imshow("Cropped face", face)
             
        if cv2.waitKey(1)==13 or int(img_id)==200: #13 is the ASCII character of Enter
            break
             
    cap.release()
    cv2.destroyAllWindows()
    print("Collecting samples is completed....")
generate_dataset()

<>:9: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:9: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\DELL\AppData\Local\Temp\ipykernel_1832\898152363.py:9: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


In [4]:
!pip install pillow

In [5]:
import os
import cv2
from PIL import Image #pip install pillow
import numpy as np    # pip install numpy
 

In [10]:
import os
os.listdir(r"C:\Users\DELL\Desktop\Face-Recognition-main\Face Recognition")


['.ipynb_checkpoints',
 'Connect to database',
 'Face Recognition.ipynb',
 'GUI Face Recognition.ipynb',
 'haarcascade_frontalface_default.xml']

In [9]:
# Uninstall any opencv packages first (from current env)
!pip uninstall -y opencv-python opencv-contrib-python

# Then install into your user site-packages (no admin)
!pip install --user opencv-contrib-python


Found existing installation: opencv-python 4.12.0.88
Uninstalling opencv-python-4.12.0.88:
  Successfully uninstalled opencv-python-4.12.0.88


ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\DELL\anaconda3\lib\site-packages\pip\_internal\cli\base_command.py", line 160, in exc_logging_wrapper
    status = run_func(*args)
  File "C:\Users\DELL\anaconda3\lib\site-packages\pip\_internal\commands\uninstall.py", line 103, in run
    uninstall_pathset.commit()
  File "C:\Users\DELL\anaconda3\lib\site-packages\pip\_internal\req\req_uninstall.py", line 424, in commit
    self._moved_paths.commit()
  File "C:\Users\DELL\anaconda3\lib\site-packages\pip\_internal\req\req_uninstall.py", line 277, in commit
    save_dir.cleanup()
  File "C:\Users\DELL\anaconda3\lib\site-packages\pip\_internal\utils\temp_dir.py", line 173, in cleanup
    rmtree(self._path)
  File "C:\Users\DELL\anaconda3\lib\site-packages\pip\_vendor\tenacity\__init__.py", line 328, in wrapped_f
    return self(f, *args, **kw)
  File "C:\Users\DELL\anaconda3\lib\site-packages\pip\_vendor\tenacity\__init__.py", line 408, in __call__
    do = self.iter(r

     ---------------------------------------- 45.3/45.3 MB 4.0 MB/s eta 0:00:00


In [6]:
def train_classifier(data_dir):
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
     
    faces = []
    ids = []
     
    for image in path:
        img = Image.open(image).convert('L')
        imageNp = np.array(img, 'uint8')
        id = int(os.path.split(image)[1].split(".")[1])
         
        faces.append(imageNp)
        ids.append(id)
         
    ids = np.array(ids)
     
    # Train and save classifier
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces,ids)
    clf.write("classifier.xml")
train_classifier("data")

In [7]:
import cv2
import numpy as np
from PIL import Image
import os

In [ ]:
import os
import cv2
import numpy as np
from PIL import Image  # Ensure Pillow is installed with 'pip install pillow'

def train_classifier(data_dir):
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
    faces = []
    ids = []

    for image_path in path:
        img = Image.open(image_path).convert('L')  # Convert to grayscale
        image_np = np.array(img, 'uint8')
        id = int(os.path.split(image_path)[1].split('.')[1])
        faces.append(image_np)
        ids.append(id)

    ids = np.array(ids)

    # Train and save classifier
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces, ids)
    clf.write("classifier.xml")

    
def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)
    
    for (x, y, w, h) in features:
        cv2.rectangle(img, (x, y), (x+w, y+h), color, 2)
        id, _ = clf.predict(gray_img[y:y+h, x:x+w])
        cv2.putText(img, text, (x, y-4), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
    return img

def detect_face():
    faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.read("classifier.xml")
    
    video_capture = cv2.VideoCapture(0)
    
    if not video_capture.isOpened():
        print("Error: Could not open video device.")
        return
    
    while True:
        ret, img = video_capture.read()
        
        if not ret:
            print("Failed to capture image")
            break
        
        img = draw_boundary(img, faceCascade, 1.3, 6, (255, 255, 255), "Face", clf)
        cv2.imshow("Face Detection", img)
        key = cv2.waitKey(1)
        if key == 13 or key == 27:  # Enter OR ESC
            break

       
    
    video_capture.release()
    cv2.destroyAllWindows()

    
# First, train the classifier (this step should be done only once with the prepared dataset)
# train_classifier("data")

# Then, run the face detection
detect_face()


In [2]:
import tkinter as tk
from tkinter import messagebox

In [4]:
window = tk.Tk()
window.title("Face Recognition system")
 
l1 = tk.Label(window, text="Name", font=("Algerian",20))
l1.grid(column=0, row=0)
t1 = tk.Entry(window, width=50, bd=5)
t1.grid(column=1, row=0)
 
l2 = tk.Label(window, text="Age", font=("Algerian",20))
l2.grid(column=0, row=1)
t2 = tk.Entry(window, width=50, bd=5)
t2.grid(column=1, row=1)
 
l3 = tk.Label(window, text="Address", font=("Algerian",20))
l3.grid(column=0, row=2)
t3 = tk.Entry(window, width=50, bd=5)
t3.grid(column=1, row=2)
 
 
b1 = tk.Button(window, text="Training", font=("Algerian",20),bg="orange",fg="red")
b1.grid(column=0, row=4)
 
b2 = tk.Button(window, text="Detect the faces", font=("Algerian",20), bg="green", fg="orange")
b2.grid(column=1, row=4)
 
b3 = tk.Button(window, text="Generate dataset", font=("Algerian",20), bg="pink", fg="black")
b3.grid(column=2, row=4)
 
window.geometry("800x200")
window.mainloop()